# General debt projection v.4

The objetive of debt projection, as its name says, is to make a projection or aproximation of the payment needed to liquidate the debt during the current month. The projection works based on the behavior of similar debts during recent months. Particularly, the payment projection will be based on the behavior of debts of the same bank and with the same deliquency time during the last twelve months. The projection will be presented as a discount over the actual balance of the debt. As we don't know the actual balance of the debt, we will need to make an aproximation of that too.

We have two metods for discount projection. Noah method uses the last discount the debt got and projects the new discount as the next discount on the list. The list is created based on the behavior of the debts. Percentil method uses the 15% percentil discount curve for each deliquency time and projects the new discount as the corresponding discount depending on the created curve. Noah method will be the main method, while percentil method will only be used on debts with no history of discounts.

The process of projection will follow the next steps:

1. Establishing parameters for the algorithm.
2. Import and clean data.
3. Actual balance aproximation.
4. Logarithmic discount curve adjustment.
5. Use of the curves for discount projection
    1. Noah Method.
    2. Percentil Metod.
6. Saving results.

## Parameters of the algorithm

In [1]:
# Libraries
import pandas as pd # for dataframes
import numpy as np # for numerical processes
import matplotlib.pyplot as plt # for graphs
from sklearn.metrics import mean_squared_error

In [2]:
# settings
pd.set_option('display.max_columns', 100) # displays 100 columns on dataframes

In [3]:
# parameters
activities_source = 'BBVA activities apr23-mar24.csv' # activities are the history of discounts
bins_source = 'BBVA_bins.csv' # bins indicate the interest rate of each debt
results_file_name = 'descuentos_proyectados_BBVA.csv'
result_curve1_file_name = 'curva_descunto_BBVA.csv'
bank_input = 'card' # card or loan (only for BBVA and Santander, for other banks write False)
max_period = 18 #max period of interest grow
max_discount = 0.85 # max discount for projections
default_discount = 0.75 # default discount for bigger deliquencies than curve size

In [4]:
# advanced parameters
date_format = '%Y-%m-%d' # for formatting on date
default_interest_rate = 0.04
bigger_tolerance_rate = 3 # max times payment can be bigger than debt. bigger will be considered a mistake.
max_discount_tolerance = 0.98 #max discount. bigger will be considered a mistake.
curve_size = 20 # deliq months to consider in curves
percentile_1 = 15 # smaller percentil for percentil method
percentile_2 = 50 # bigger percentil for Noah metod
today = np.datetime64('today') #analysis date

In [5]:
# gradient descent method parameters
a0 = [0.4,10] # initial parameters
a1 = a0 + np.random.rand(2)*[0.1,1] # initial parameters
iterations = 100
step = 0.01
parameter_range = 4 # discrete steps to try for the beggining of the log function

## Data

In [6]:
#import data
activities = pd.read_csv(activities_source)
bins = pd.read_csv(bins_source)

In [7]:
# clean activities
activities.inicio_programa = pd.to_datetime(activities.inicio_programa,format=date_format,errors='coerce') # errors in cast will be NaT
activities.ultimo_pago = pd.to_datetime(activities.ultimo_pago,format=date_format,errors='coerce')
activities.executed_date = pd.to_datetime(activities.executed_date,format=date_format,errors='coerce')
activities[['bank_reference','debt_id','banco','num_tarjeta']] = activities[['bank_reference','debt_id','banco','num_tarjeta']].astype(str) #simultaneous cast
activities.dropna(inplace=True) #drop rows with null or NaT values
activities.reset_index(drop = True,inplace=True)

In [8]:
#clean Bins
bins['Bin'] = bins['Bin'].astype(str).str.zfill(6) # zfill ensures data has 6 digits, fills with zeros.
bins.drop_duplicates('Bin',inplace=True)
bins.reset_index(drop = True,inplace=True)

## Balance Aproximation

In [9]:
if bank_input: # 'if' reads anything thats not 'False' or 0 as True
    cards = activities.num_tarjeta.str.replace(r'\D','',regex=True) # regex help us filter only numeric values
    cards_len = cards.str.len()
    activities['bank_type'] = np.where((cards_len >= 15)&(cards_len <= 17), 'card','loan') # cards have 16 digits
    activities['Bin'] = cards.str[:6]
    activities = activities[activities.bank_type == bank_input].reset_index(drop = True) # reset index help us keep an ordered dataframe
else:
    cards = activities.num_tarjeta.str.replace(r'\D','',regex=True)
    activities['Bin'] = cards.str[:6]

In [10]:
activities = pd.merge(activities, bins, on = 'Bin', how = 'left') # we merge to find interest rate

In [11]:
activities.tasa = activities.tasa.fillna(default_interest_rate) # we fill with default interest rate bins not find
#we use the date formula [(Y2-Y1)*12 + M2-M1] to calculate deliquencies
activities['deliq_beginning_prog'] = (activities.inicio_programa.dt.year - activities.ultimo_pago.dt.year)*12 + activities.inicio_programa.dt.month - activities.ultimo_pago.dt.month
activities.deliq_beginning_prog = np.where(activities.deliq_beginning_prog < 0, 0, activities.deliq_beginning_prog)
activities['deliq_act'] = (activities.executed_date.dt.year - activities.ultimo_pago.dt.year)*12 + activities.executed_date.dt.month - activities.ultimo_pago.dt.month
activities.drop(activities[activities.deliq_act <0].index,inplace=True) # we drop negative periods as there must be a payment after the discount
#aux_balance help us to know the deliquency months passed from last record
activities['aux_balance'] = np.where(activities.deliq_beginning_prog > max_period, 0,
                                     np.where(activities.deliq_act > max_period, max_period - activities.deliq_beginning_prog,
                                              activities.deliq_act - activities.deliq_beginning_prog))
# we use the compound interest formula for balance
activities['balance'] = activities.deuda_resuelve*(1 + activities.tasa)**(activities.aux_balance)
activities['real_discount'] = 1-activities.pago_a_bancos/activities.balance
# we filter activities to avoid incongruent payments
activities.drop(activities[activities.deuda_resuelve*bigger_tolerance_rate < activities.pago_a_bancos].index, inplace = True)
activities.drop(activities[activities.real_discount > max_discount_tolerance].index,inplace=True)
activities.reset_index(drop = True,inplace=True)

## Curve Adjustment

### Gradent Descent Method

we'll use gradent descent metod to make an adjusted logarithmic curve that passess through the percentil points

In [12]:
#gradient descent function
def grad_desc(f,x0,x1,iterations,f_step):
    # this is the gradent descent method for R^2 -> R functions that uses
    # numerical aproximations of the derivate of the function.
    xii = np.array(x0) # numpy arrays are better for math ops
    xjj = np.array(x1) # j = i + 1
    it = 0
    while it < iterations and any(xii != xjj):
        xji = np.array([xjj[0],xii[1]])
        xij = np.array([xii[0],xjj[1]])
        if xjj[0] == xii[0]:
            df_dx = 0
        else:
            df_dx = (f(xji)-f(xii)) / (xjj[0]-xii[0])
        if xjj[1] == xii[1]:
            df_dy = 0
        else:
            df_dy = (f(xij)-f(xii)) / (xjj[1]-xii[1])
        mov = np.array([df_dx,df_dy]) * (-f_step)
        xii = xjj
        xjj = xjj + mov
        it = it + 1
    return xjj,f(xjj)

### Curve Adjustment

In [13]:
c1, c2 = np.zeros(curve_size), np.zeros(curve_size) # initialize adjusted curves
# basic percentile assignation to the curves
for month in range(1,curve_size):
    c1[month] = np.percentile(activities[activities.deliq_act == month].real_discount,percentile_1)
    c2[month] = np.percentile(activities[activities.deliq_act == month].real_discount,percentile_2)
#finding ideal parameters for the curves
for curve in [c1,c2]:
    for log_step in range(parameter_range):
        # f is the logarithmic function we will use
        def f(a,x):
            if (x-log_step)*a[1] <= 0:
                res = 0
            else:
                res = max(0,a[0]*np.log((x-log_step)*a[1]))
            return res
        # we use Mean Squared Error to evaluate error magnitude
        def MSE(a): return mean_squared_error([f(a,x) for x in range(curve_size)],curve)
        # we use grad desc to find optimal parameters
        a_candidate,f_candidate = grad_desc(MSE,a0,a1,iterations,step)
        # if we dont have an optimal value yet, use the first one
        try:
            if f_candidate < f_star:
                a_star,f_star,log_step_star = a_candidate,f_candidate,log_step
        except:
            a_star,f_star,log_step_star = a_candidate,f_candidate,log_step
    def f(a,x):
        if (x-log_step_star)*a[1] <= 0:
            res = 0
        else:
            res = max(0,a[0]*np.log((x-log_step_star)*a[1]))
        return res
    if all(curve == c1):
        c1_adjusted = pd.DataFrame([f(a_star,x) for x in range(curve_size)],columns=['discount'])
    else:
        c2_adjusted = pd.DataFrame([f(a_star,x) for x in range(curve_size)],columns=['discount'])
    del f_star # we delete it so its not saved for the next loop

## Discount Projection

In [14]:
# we create a dataframe 'debts' to filter info by debt
debts = activities.sort_values(['debt_id','executed_date'],ascending=[True,False]).drop_duplicates(["debt_id"]).copy()
debts = debts[['bank_reference','debt_id','ultimo_pago','deliq_beginning_prog','deuda_resuelve','tasa','real_discount','pago_a_bancos']].reset_index(drop = True)
debts.rename(columns = {'real_discount':'last_discount','pago_a_bancos':'last_payment'},inplace=True)
today = pd.to_datetime(today)
debts['deliq_today'] = (today.year - debts.ultimo_pago.dt.year)*12 + today.month - debts.ultimo_pago.dt.month
debts['aux_balance_today'] = np.where(debts.deliq_beginning_prog > max_period, 0,
                                np.where(debts.deliq_today > max_period, max_period - debts.deliq_beginning_prog,
                                         debts.deliq_today - debts.deliq_beginning_prog))
debts["balance_today"] = debts.deuda_resuelve*(1+debts.tasa)**(debts.aux_balance_today)
#Now we apply Noah algorithm
for indx in debts.index:
    if debts.loc[indx,'last_discount'] > max_discount:
        debts.loc[indx,'projected_discount'] = max_discount
    else:
        debts.loc[indx,'projected_discount'] = c2_adjusted.discount[c2_adjusted.discount > debts.loc[indx,'last_discount']].head(1).values
debts['projected_payment'] = debts.balance_today * (1-debts.projected_discount)
# as a final step, we will load only payments as good as the last one
debts['load_adjustment'] = np.where(debts.projected_payment < debts.last_payment,
                                   debts.projected_payment, debts.last_payment)                                   

## Saving Results

In [15]:
debts.to_csv(results_file_name)
c1_adjusted.to_csv(result_curve1_file_name)